In [201]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [202]:
maxlen = 20
labels = 2

In [203]:
df = pd.read_csv('name_gender_dataset.csv', header=None)
df.columns = ['name', 'gender', 'count', 'probability']


columnsTitles=['name', 'gender', 'count', 'probability']
df = df.reindex(columns=columnsTitles)
del df['count']
del df['probability']

names = df['name']
gender = df['gender']

names1 = [x.lower() for x in names]
token = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n')
token.fit_on_texts(names)
token.get_config()

df.head

<bound method NDFrame.head of             name  gender
0           Name  Gender
1          James       M
2           John       M
3         Robert       M
4        Michael       M
...          ...     ...
9995  Martavious       M
9996      Vander       M
9997     Krystel       F
9998  Nicollette       F
9999       Elson       M

[10000 rows x 2 columns]>

In [204]:
print("Male : " + str(sum(gender=='M')))
print("Female : " + str(sum(gender=='F')))

Male : 4037
Female : 5962


In [205]:
names = df['name']
gender = df['gender']

names1 = [x.lower() for x in names]
token = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n')
token.fit_on_texts(names)
token.get_config()

vocab = set(' '.join([str(i) for i in names1]))
vocab.add('END')

len_vocab = len(vocab)
print(len_vocab)
print(vocab)

29
{'o', 'w', 'm', 'j', 'a', 'h', 'p', 'k', 'l', 'd', 'f', 'v', 'z', 'r', 'y', 'g', 's', '-', ' ', 'n', 'q', 'b', 'i', 'c', 'x', 'e', 't', 'END', 'u'}


In [206]:
char_index = dict((c, i) for i, c in enumerate(vocab))
char_index

{'o': 0,
 'w': 1,
 'm': 2,
 'j': 3,
 'a': 4,
 'h': 5,
 'p': 6,
 'k': 7,
 'l': 8,
 'd': 9,
 'f': 10,
 'v': 11,
 'z': 12,
 'r': 13,
 'y': 14,
 'g': 15,
 's': 16,
 '-': 17,
 ' ': 18,
 'n': 19,
 'q': 20,
 'b': 21,
 'i': 22,
 'c': 23,
 'x': 24,
 'e': 25,
 't': 26,
 'END': 27,
 'u': 28}

In [207]:
msk = np.random.rand(len(df)) < 0.8
train = (df[msk])
test = (df[~msk])

In [208]:
def set_flag(i):
    tmp = np.zeros(29);
    tmp[i] = 1
    return(tmp)

In [209]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [210]:
X_train = []
Y_train = []

for i in names1:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X_train.append(tmp)
for i in gender:
    if i == 'm':
        Y_train.append([1,0])
    else:
        Y_train.append([0,1])

In [211]:
np.asarray(X_train).shape

(10000, 20, 29)

In [212]:
np.asarray(Y_train).shape

(10000, 2)

In [220]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.LSTM(512, return_sequences=False),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [214]:
X_test = []
Y_test = []

for i in names1:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X_test.append(tmp)
for i in gender:
    if i == 'm':
        Y_test.append([1,0])
    else:
        Y_test.append([0,1])

In [215]:
print(np.asarray(X_test).shape)
print(np.asarray(Y_test).shape)

(10000, 20, 29)
(10000, 2)


In [216]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [221]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=1000, epochs=10, validation_data=(X_test, Y_test))

Epoch 1/10
10/10 [==============================] - 87s 9s/step - loss: 0.2684 - accuracy: 0.7357 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
10/10 [==============================] - 82s 8s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
10/10 [==============================] - 82s 8s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
 5/10 [==============>...............] - ETA: 29s - loss: 0.0000e+00 - accuracy: 1.0000

In [219]:
score, accuracy = model.evaluate(X_test, Y_test)
print('Test Score: ', score)
print('Test Accuracy: ', accuracy)

313/313 [==============================] - 36s 112ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test Score:  0.0
Test Accuracy:  1.0
